<a href="https://colab.research.google.com/github/SATHYARANGARAJ53/medicine_inventory/blob/main/projectmodel_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from statsmodels.tsa.seasonal import seasonal_decompose
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from prophet import Prophet

# Load dataset
df = pd.read_csv("medicine_details.xlsx.csv")

# Generate synthetic stock data
df['Stock Level'] = np.random.randint(50, 500, df.shape[0])
df['Weekly Consumption'] = np.random.randint(10, 50, df.shape[0])
df['Stock Expiry (Days)'] = np.random.randint(10, 120, df.shape[0])

# CHAID for real-time stock analysis
contingency_table = pd.crosstab(df['Stock Expiry (Days)'] < 30, df['Stock Level'] < 100)
chi2, p, _, _ = chi2_contingency(contingency_table)
print(f"CHAID Test - Chi2: {chi2}, P-Value: {p}")

# LSTM for utilization trends
sequence_length = 5
X, y = [], []
for i in range(len(df) - sequence_length):
    X.append(df['Weekly Consumption'].iloc[i:i + sequence_length].values)
    y.append(df['Weekly Consumption'].iloc[i + sequence_length])
X, y = np.array(X), np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))

model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(sequence_length, 1)),
    LSTM(50, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=10, verbose=1)

# Prophet for demand forecasting
prophet_df = df[['Medicine Name', 'Weekly Consumption']].rename(columns={'Weekly Consumption': 'y'})
prophet_df['ds'] = pd.date_range(start='2025-01-01', periods=len(df), freq='W')
prophet_model = Prophet()
prophet_model.fit(prophet_df)
future = prophet_model.make_future_dataframe(periods=12, freq='W')
forecast = prophet_model.predict(future)
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())


CHAID Test - Chi2: 1.237163223390117, P-Value: 0.2660184710701173
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 1025.2404
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 863.1003
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 589.8961
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 281.5468
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 189.9546
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 172.9384
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 129.3131
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 154.8832 
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 138.2413 
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 140.6920


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpp9e66i4g/kl04yy89.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpp9e66i4g/4wi815px.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=75186', 'data', 'file=/tmp/tmpp9e66i4g/kl04yy89.json', 'init=/tmp/tmpp9e66i4g/4wi815px.json', 'output', 'file=/tmp/tmpp9e66i4g/prophet_modelsjtgql5r/prophet_model-20250131103247.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:32:47 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:32:47 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


            ds       yhat  yhat_lower  yhat_upper
127 2027-06-13  27.189705   15.259654   40.836261
128 2027-06-20  27.363223   14.877419   39.408948
129 2027-06-27  31.644218   19.805395   44.396226
130 2027-07-04  35.444668   22.671905   47.835443
131 2027-07-11  35.788553   22.930502   49.334561
